[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-durand/propedeutico_python/blob/main/notebooks/7_SQL.ipynb)

# Propedéutico a programación con Python.

**Verano 2023, por el Centro de Ciencia de Datos, EGobiernoyTP.**

## Sesión 7: SQL básico

1. Introducción
    * Bases de datos
    * Lenguaje SQL
2. Establecer conexiones con las bases de datos
    * sqlite 
    * psycopg2 
3. Acciones básicas con SQL
    * Queries
    * 
3. Búsquedas avanzadas con SQL

---
# 1 Introducción

## Bases de datos

Las bases de datos relacionales es otra manera en la que se puede almacenar información.


Generalmente, las bases de datos de esta naturaleza se guardan en servidores (computadoras en la nube) dedicados exclusivamente para el mantenimiento y manejo de datos tabulares (columna-renglón). Las bases de datos relacionales son de gran ayuda sobre todo cuando se manejan grandes cantidades de datos con una estructura muy amigable.

RDBMS stands for Relational Database Management System.

RDBMS is the basis for SQL, and for all modern database systems such as MS SQL Server, IBM DB2, Oracle, MySQL, and Microsoft Access.

The data in RDBMS is stored in database objects called tables. A table is a collection of related data entries and it consists of columns and rows.

Every table is broken up into smaller entities called fields. The fields in the Customers table consist of CustomerID, CustomerName, ContactName, Address, City, PostalCode and Country. A field is a column in a table that is designed to maintain specific information about every record in the table.

A record, also called a row, is each individual entry that exists in a table. For example, there are 91 records in the above Customers table. A record is a horizontal entity in a table.

A column is a vertical entity in a table that contains all information associated with a specific field in a table.

A database most often contains one or more tables. Each table is identified by a name (e.g. "Customers" or "Orders"). Tables contain records (rows) with data.


---

## Lenguaje SQL

Un lenguaje estándar que se utiliza para interactuar con estas bases es SQL (Structured Query Language). Con SQL puedes almacenar, manipular y recuperar información en la tabla o sobre ella. Específicamente, además de crear bases de datos te permite hacer las siguientes actividades con una base de datos:

* hacer búsquedas 
* obtener datos
* insertar datos
* actualizar datos
* borrar elementos (renglones/columnas/tablas)
* crear tablas 
* automatizar procedimientos almacenados
* definir permisos sobre tablas y procedimientos


# 2 Establecer conexiones con las bases de datos

Algunos paquetes para interactuar entre python y bases de datos relacionales son `psycopg2` y `sqlite`, ambos utilizan SQL. El que se utilizará más durante la maestría es `psycopg2` y lo usarás para interactuar con servidores. Sin embargo, para enseñarte el funcionamiento de SQL aquí usaremos bases de datos locales y utilizaremos `sqlite`.

---

## Establecer conexión con sqlite

Primero cargamos la librería:

In [18]:
import sqlite3

Para fines de la sesión de hoy utilizamos una base de datos local para mostrar cómo puedes cargar datos con sql. 
La base se llama `flights` y está en la carpeta datos:

In [2]:
gdrv_file = '../datos/flights.db'

Después generas una conexión con la base de datos dicha conexión permite a Python saber en dónde se encuentra la base con la que estarás trabajando. Si no existiera una base con ese nombre entonces `sqlite3` la crearía implícitamente.

In [9]:
con = sqlite3.connect(gdrv_file)
con

## Establecer conexión a través de psycopg2

En el caso de psycopg2, para establecer la conexión a bases de datos (del tipo PSQL) necesitas las siguientes líneas de código, así como la información que se enumera:


```
# Importas paquete
import psycopg2

# Estableces conexión: 
psconn = psycopg2.connect(host="mouse.db.elephantsql.com", # 1. Dirección de la base
                          port = 5432,                     # 2. Puerto de conexión
                          database="base_mouse",           # 3. Nombre de la base
                          user="usuario-n",                # 4. Nombre de usuario
                          password="your-password")        # 5. Contraseña
                          

```


La información numerada se asigna y proporciona generalmente por el administrador/dueño de la base de datos. Esto es para tener mayor seguridad y control de la información.

---


# 3 Acciones básicas con SQL

Una vez que tienes una conexión entre Python y la base de datos, puedes utilizar el lenguaje SQL para interactuar con la tabla `airlines` de la base de datos `flights`, a través de una instrucción de SQL llamada **_query_**: 

In [10]:
q = '''
    SELECT 
        * 
    FROM 
        airlines
        ;
    '''

``q`` es una _query_, es simplemente una instrucción (aquí lo guardamos en una cadena de texto de Python) que puede tener distintas palabras clave (en ese caso están escritas con mayúsculas) que realizan acciones sobre una base de datos o una tabla. El ejemplo ``q`` anterior hace lo siguiente:

- `SELECT` : selecciona
- `*` : todo
- `FROM` : de 
- `airlines` : tabla llamada airlines
- `;` : así se cierran los queries en sql

Así, en la siguiente celda, con ``q`` obtendremos todos los datos de la tabla a este notebook. Lo haremos a través de la conexión recién creada, y para que el formato de las interacciones con la tabla sea estético utilizaremos `pandas` (se puede cargar con un formato más simple utilizando sólo `sqlite`).

Las _queries_ se pueden ejecutar a través de Python -como lo mostraremos en este tutorial- o a través de una terminal sin requerir Python.

In [20]:
import pandas as pd

In [21]:
df = pd.read_sql(q, con)
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N
...,...,...,...,...,...,...,...,...,...
6043,6043,19828,Vuela Cuba,Vuela Cuba,6C,6CC,None,Cuba,Y
6044,6044,19830,All Australia,All Australia,88,8K8,None,Australia,Y
6045,6045,19831,Fly Europa,None,ER,RWW,None,Spain,Y
6046,6046,19834,FlyPortugal,None,PO,FPT,FlyPortugal,Portugal,Y


Some of The Most Important SQL Commands

`SELECT` - extracts data from a database
`UPDATE` - updates data in a database
`DELETE` - deletes data from a database
`INSERT INTO` - inserts new data into a database
`CREATE DATABASE` - creates a new database
`ALTER DATABASE` - modifies a database
`CREATE TABLE` - creates a new table
`ALTER TABLE` - modifies a table
`DROP TABLE` - deletes a table
`CREATE INDEX` - creates an index (search key)
`DROP INDEX` - deletes an index

## Observaciones

* Las palabras clave NO distinguen entre mayúsculas y minúsculas. Por ejemplo, `select` es equivalente a `SELECT`. Pero es recomendable escribir en mayúscula pues el estándar en varios proyectos.

* El punto y coma se escribe pues indica que un _query_ acaba y permite separarlo de otros y ejecutarlos en una misma llamada.


